En este cuaderno, vamos a probar a usar el modelo scGPT a través de la plataforma HugginFace. Trataremos de seguir con este los tutoriales ofrecidos por los investigadores, aunque estos se realizaron pensado en la instalación completa del modelo. Veremos hasta que punto es compatible usar el modelo a través de Huggin Face.

In [ ]:
from tdc.multi_pred.anndata_dataset import DataLoader
from tdc import tdc_hf_interface
from tdc.model_server.tokenizers.scgpt import scGPTTokenizer
import torch

# Dataset de ejemplo, 36406 columnas y 4992 instancias
# adata es un objeto de tipo AnnData(Annotated Data)
adata = DataLoader("cellxgene_sample_small",
                   "./data",
                   dataset_names=["cellxgene_sample_small"],
                   no_convert=True).adata

# Estas dos primeras líneas cargan el modelo
scgpt = tdc_hf_interface("scGPT")
model = scgpt.load()

# Montamos la estructura de datos necesaria para los embeddings
tokenizer = scGPTTokenizer()

#Creamos un array con los nombres de los genes(id's)
gene_ids = adata.var["feature_name"].to_numpy()

# Convierte los perfiles de transcripcion a tokens
tokenized_data = tokenizer.tokenize_cell_vectors(
    adata.X.toarray(), gene_ids)

#Máscara para los valores de expresión iguales a 0
mask = torch.tensor([x != 0 for x in tokenized_data[0][1]], #¿Por qué el primero siempre es un 0?
                    dtype=torch.bool)

# Embedding de la primera tupla (gene_id, count)
# los embeddings son generados por medio del modelo
# en este caso, sin hacer fine-tuning

first_embed = model(tokenized_data[0][0], #Primer elemento de la primera tupla
                    tokenized_data[0][1], #Segundo elemento de la primera tupla
                    attention_mask=mask)


Found local copy...
Loading...
Found local copy...


In [3]:
print(type(model))

<class 'tdc.model_server.models.scgpt.ScGPTModel'>


In [2]:
### Entendiendo el formato AnnData, Documentación oficial ###
###                                                       ###

# Todos los nombre de variables que salen son metadatos
adata.obsm["spatial"]

# Representación de los datos como una matriz dispersa
# La matriz tiene sus columnas para las observaciones y las variables, de forma diferenciada
#adata.X

# Acceso mediante índices
#print("----- ID de las filas -----")
#print(adata.obs_names) # Transcriptomas(filas)
#print("----- ID de las columnas -----")
#print(adata.var_names) # Genes(columnas)

# Podemos visualizarlo como un DataFrame
# Fíjate que hay columnas cuyos valores son "nulos" casi en su totalidad
# .obs te devuelve información sobre las observaciones(transcritos)
#adata.obs

# Tanto .obs como .var tienen el formato Dataframe(se pueden convertir al de pandas)
# .var te devuelve información sobre las variables(genes)
#adata.var["feature_name"].head(10)

#Convertimos la matriz a un array de las expresiones
#adata.X.toarray()

array([[1129.32443367, 8124.38778062],
       [6053.00411081, 3281.33693547],
       [1427.00474023, 6600.4981705 ],
       ...,
       [5553.76652213, 7515.25536911],
       [8307.49301103, 6732.16484772],
       [1821.01498753, 6149.80761267]])

In [ ]:
# ¿Qué hace el tokenizer?
# tokenized_data es una lista de tuplas, de dimensión la cantida de obs de adata
# cada elemento de la tupla es un tensor de pytorch(vectores numéricos)
tokenized_data

In [ ]:
# El embedding se guarda en un diccionario
import torch
from anndata import AnnData

first_embed
#embed_adata = AnnData()

#embedding_2D = torch.stack(list(first_embed.values()))
#embedding_2D.shape

In [ ]:
### Representamos los embeddings usando scanpy

import scanpy as sc
import matplotlib.pyplot as plt
import numpy



sc.pp.neighbors(adata)

In [ ]:
import copy
import gc
import json
import os
from pathlib import Path
import shutil
import sys
import time
import traceback
from typing import List, Tuple, Dict, Union, Optional
import warnings
import pandas as pd
# from . import asyn
import pickle
import torch
from anndata import AnnData
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import wandb
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from sklearn.metrics import confusion_matrix

sys.path.insert(0, "../")
import scgpt as scg
from scgpt.model import TransformerModel, AdversarialDiscriminator
from scgpt.tokenizer import tokenize_and_pad_batch, random_mask_value
from scgpt.loss import (
    masked_mse_loss,
    masked_relative_error,
    criterion_neg_log_bernoulli,
)
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.preprocess import Preprocessor
from scgpt import SubsetsBatchSampler
from scgpt.utils import set_seed, category_str2int, eval_scib_metrics

sc.set_figure_params(figsize=(6, 6))
os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')